In [ ]:

from numpy import *
set_printoptions(legacy = '1.25')

def f(x): return sin(x)
def g(r): return 1/(1+ exp(-r))
def h(s): return s**2

functions = [f,g,h]

def df(x): return cos(x)
def dg(r): return g(r)*(1-g(r))
def dh(s): return 2*s

derivatives = [df,dg,dh]


In [ ]:


# first version: chains

def forward_prop(x_in, functions):
	x = [x_in]
	for f in functions:
		x_out = f(x_in)
		x.append(x_out) # insert at end
		x_in = x_out
	return x

from numpy import *
x_in = pi/4
x = forward_prop(x_in, functions)
x


In [ ]:

# dy/dy = 1
delta_out = 1


In [ ]:


# first version: chains

def backward_prop(delta_out, x, derivatives):
	delta = [delta_out]
	rev = reversed
	# discard last element then reverse x
	# also reverse derivatives
	for a, df in  zip(rev(x[:-1]), rev(derivatives)):
		# chain rule -- multiply by previous der
		der = df(a) * delta[0]
		delta.insert(0,der) # insert at start
	return delta
	
delta = backward_prop(delta_out, x, derivatives) 
delta


In [ ]:


d = 3
functions, derivatives = [h]*d, [dh]*d
x_in, delta_out = 5, 1

x = forward_prop(x_in, functions)
delta = backward_prop(delta_out, x, derivatives) 
x, delta


In [ ]:

d = 7
w = [ [0]*d for _ in range(d) ] 

# indexing in code starts at 0
# nodes are 0, 1, 2, 3, 4, 5, 6

w[3][0] = w[3][1] = w[4][1] = w[4][2] = 1
w[5][3] = w[5][4] = w[6][5] = 1


In [ ]:

activate = [0]*d

# indexing starts at 0

activate[3] = lambda x,y: x+y
activate[4] = lambda y,z: max(y,z)
activate[5] = lambda a,b: a*b


In [ ]:

def incoming(x, w, i):
	return [ w[i][j] * outgoing(x, w, j) for j in range(d) if w[i][j] ]


In [ ]:

def outgoing(x, w, i):
	if x[i] != None: return x[i]
	elif activate[i]: return activate[i](*incoming(x, w, i))
	else: return None


In [ ]:


# second version: networks

def forward_prop(x_in, w):
	d = len(w)
	x = [None]*d
	m = len(x_in)
	x[:m] = x_in
	for i in range(m,d): x[i] = outgoing(x, w, i)
	return x
	
x_in = [1,2,0]
x = forward_prop(x_in, w)
x


In [ ]:


g = [ [0]*d for _ in range(d) ]

# g specified only at neurons

# indexing starts at 0

g[3][0] = lambda x,y: 1
g[3][1] = lambda x,y: 1
g[4][1] = lambda y,z: 1 if y >= z else 0
g[4][2] = lambda y,z: 1 if z > y else 0
g[5][3] = lambda a,b: b
g[5][4] = lambda a,b: a


In [ ]:


def derivative(x, delta, g, j):
	if delta[j] != None: return delta[j]
	else:
		return sum([ derivative(x, delta, g, i) * g[i][j]( *incoming(x, w, i)) * w[i][j] for i in range(d) if w[i][j] ] )


In [ ]:


# second version: networks

def backward_prop(x, delta_out, g):
	d = len(g)
	delta = [None]*d
	m = len(delta_out)
	delta[-m:] = delta_out
	for j in range(d-m): 
		delta[j] = derivative(x, delta, g, j)
	return delta

delta_out = [1,None]
delta = backward_prop(x, delta_out, g)
delta
